# Setup

## Bilinear pairing and generator

In [42]:
def modulo_divide(p, q):
    p = int(p % 173)
    q = int(q % 173)
    for i in range(1, 173):
      if (i * q) % 173 == p:
        return i
modulo_divide(41, 92)

149

In [43]:
# Elliptic Curve

class Point_on_EC:
  def __init__(self, x, y):
    self.x = x
    self.y = y

class O:
  def __init__(self):
    self.x = "inf"
    self.y = "inf"

import math
import random
import warnings
import numpy as np
import cmath

class Elliptic_curve:
  def __init__(self, A, B, p):
    self.A = A
    self.B = B
    self.p = p
    disc_check = (4*(self.A**3) + 27*(self.B**2)) % self.p
    if disc_check == 0:
      warnings.warn("The discriminant of elliptic curve is 0, some functions may raise Error.")
  
  def discriminant(self):
    return (4*(self.A**3) + 27*(self.B**2)) % self.p
  
  # def plot(self):
  #   a = self.A
  #   b = self.B

  #   y, x = np.ogrid[-self.p:self.p:200j, -self.p:self.p:200j]
  #   plt.contour(x.ravel(), y.ravel(), pow(y, 2) - pow(x, 3) - x * a - b, [0])
  #   plt.grid()
  #   plt.show()
  
  def __modulo_divide(self, p, q):
    p = int(p % self.p)
    q = int(q % self.p)
    for i in range(1, self.p):
      if (i * q) % self.p == p:
        return i
  
  def addition(self, P, Q):
    if self.discriminant() == 0:
      raise ValueError("Discriminant of elliptic curve is 0, please change value of 'A' and 'B' in 'y^2 = x^3 + A*x + b.")
    
    if isinstance(P, O):
      return Q
    elif isinstance(Q, O):
      return P
    elif P.x == Q.x and P.y == Q.y:
      if isinstance(P.x, complex) or isinstance(P.y, complex):
        l = self.complex_divide(complex((3*self.complex_multiply(P.x, P.x) + self.A).real % self.p, (3*self.complex_multiply(P.x, P.x)).imag % self.p), complex((2*P.y).real % self.p, (2*P.y).imag % self.p))
      else:
        l = self.__modulo_divide((3*(P.x*P.x) + self.A) % self.p, (2*P.y) % self.p)
    elif P.x == Q.x and (P.y.real == ((-Q.y).real % self.p) and P.y.imag == ((-Q.y).imag % self.p)):
      return O()
    else:
      if isinstance(P.x, complex) or isinstance(P.y, complex) or isinstance(Q.x, complex) or isinstance(Q.y, complex):
        l = self.complex_divide(complex((Q.y-P.y).real % self.p, (Q.y-P.y).imag % self.p), complex((Q.x-P.x).real % self.p, (Q.x-P.x).imag % self.p))
      else:
        l = self.__modulo_divide((Q.y-P.y) % self.p, (Q.x-P.x) % self.p)
    
    if isinstance(l, complex):
      x3 = complex((self.complex_multiply(l, l) - P.x - Q.x).real % self.p, (self.complex_multiply(l, l) - P.x - Q.x).imag % self.p)
      y3 = complex((self.complex_multiply(l, (P.x-x3)) - P.y).real % self.p, (self.complex_multiply(l, (P.x-x3)) - P.y).imag % self.p)
    else:
      x3 = (l*l - P.x - Q.x) % self.p
      y3 = (l*(P.x-x3) - P.y) % self.p
    return Point_on_EC(x3, y3)
  
  def subtraction(self, P):
    if self.discriminant() == 0:
      raise ValueError("Discriminant of elliptic curve is 0, please change value of 'A' and 'B' in 'y^2 = x^3 + A*x + b.")
    # print(P)
    # print(P.x, P.y)
    x = P.x

    if isinstance(P.y, complex):
      y_real = -P.y.real % self.p
      y_imag = -P.y.imag % self.p
      y = complex(y_real, y_imag)
    else:
      y = (-P.y) % self.p
    return Point_on_EC(x, y)
  
  def multiply(self, P, n):
    if self.discriminant() == 0:
      raise ValueError("Discriminant of elliptic curve is 0, please change value of 'A' and 'B' in 'y^2 = x^3 + A*x + b.")
    # Multiplying a point on Elliptic Curve using the Double-and_Add Algorithm
    Q = P
    R = O()
    # i = 0
    # print(i)
    # print(n)
    # print('Q', Q.x, Q.y)
    # print('R', R.x, R.y)
    # print("----------------")
    while n > 0:
      if (n % 2) == 1:
        R = self.addition(R, Q)
      Q = self.addition(Q, Q)
      n = math.floor(n/2)
      # i += 1
      # print(i)
      # print(n)
      # print('Q', Q.x, Q.y)
      # print('R', R.x, R.y)
      # print("----------------")
    return R
  
  def find_order(self, P):
    i = 1
    R = P
    # print(R.x, R.y)
    while not isinstance(R, O):
      # print(R.x, R.y)
      R = self.addition(P, R)
      i += 1
      if i > self.p:
        return False
    return i
    
  def __g_P_Q(self, P, Q, R):
    if isinstance(P, O):
      return Q
    elif isinstance(Q, O):
      return P
    elif P.x == Q.x and P.y == Q.y:
      if isinstance(P.x, complex) or isinstance(P.y, complex):
        l = self.complex_divide(complex((3*self.complex_multiply(P.x, P.x) + self.A).real % self.p, (3*self.complex_multiply(P.x, P.x)).imag % self.p), complex((2*P.y).real % self.p, (2*P.y).imag % self.p))
      else:
        l = self.__modulo_divide((3*(P.x*P.x) + self.A) % self.p, (2*P.y) % self.p)
    elif P.x == Q.x and (P.y.real == ((-Q.y).real % self.p) and P.y.imag == ((-Q.y).imag % self.p)):
      l = 'inf'
    else:
      if isinstance(P.x, complex) or isinstance(P.y, complex) or isinstance(Q.x, complex) or isinstance(Q.y, complex):
        l = self.complex_divide(complex((Q.y-P.y).real % self.p, (Q.y-P.y).imag % self.p), complex((Q.x-P.x).real % self.p, (Q.x-P.x).imag % self.p))
      else:
        l = self.__modulo_divide((Q.y-P.y) % self.p, (Q.x-P.x) % self.p)

    # print((3*(P.x*P.x) + self.A))
    # print((3*(P.x*P.x) + self.A) % self.p)
    # print((2*P.y) % self.p)
    # print('l', l)

    if l != "inf":
      if isinstance(l, complex) or any(isinstance(a, complex) for a in [P.x, P.y, Q.x, R.x, R.y]):
        return self.complex_divide(complex((R.y - P.y - self.complex_multiply(l, (R.x - P.x))).real % self.p, (R.y - P.y - self.complex_multiply(l, (R.x - P.x))).imag % self.p), complex((R.x + P.x + Q.x - self.complex_multiply(l, l)).real % self.p, (R.x + P.x + Q.x - self.complex_multiply(l, l)).imag % self.p))
      else:
        return self.__modulo_divide((R.y - P.y - l*(R.x - P.x)) % self.p, (R.x + P.x + Q.x - (l*l)) % self.p)
    else:
      if isinstance(l, complex) or any(isinstance(a, complex) for a in [P.x, P.y, Q.x, R.x, R.y]):
        return complex((R.x - P.x).real % self.p, (R.x - P.x).imag % self.p)
      else:
        return (R.x - P.x) % self.p
  
  def __millers_algo(self, order, P, R):
    T = P
    f = 1
    m = bin(order)[2:]
    for i in range(1, len(m)):
      if isinstance(f, complex) or isinstance(self.__g_P_Q(T, T, R), complex):
        f = self.complex_multiply(self.complex_multiply(f, f), self.__g_P_Q(T, T, R))
      else:
        f = (((f*f) % self.p) * self.__g_P_Q(T, T, R)) % self.p
      # print('gtt', self.__g_P_Q(T, T, R))
      # print('f', f)
      T = self.addition(T, T)
      # print('T', T.x, T.y)
      # print('P', P.x, P.y)
      # print('mi', m[i])
      if int(m[i]) == 1:
        if isinstance(f, complex) or isinstance(self.__g_P_Q(T, P, R), complex):
          # print('aaa', self.__g_P_Q(T, P, R))
          f = self.complex_multiply(f, self.__g_P_Q(T, P, R))
        else:
          f = (f * self.__g_P_Q(T, P, R)) % self.p
        # print('gtp1', self.__g_P_Q(T, P, R))
        # print('f1', f)
        T = self.addition(T, P)
    #     print('T1', T.x, T.y)
    # print('----------')
    return f
  
  #https://rosettacode.org/wiki/Jacobi_symbol
  def __jacobi(self, a):
    p = self.p
    if p <= 0:
        raise ValueError("'n' must be a positive integer.")
    if p % 2 == 0:
        raise ValueError("'n' must be odd.")
    a %= p
    result = 1
    while a != 0:
        while a % 2 == 0:
            a /= 2
            n_mod_8 = p % 8
            if n_mod_8 in (3, 5):
                result = -result
        a, p = p, a
        if a % 4 == 3 and p % 4 == 3:
            result = -result
        a %= p
    if p == 1:
        return result
    else:
        return 0
  
  def __power(self, x, y): 
    res = 1
    x = x % self.p
 
    while (y > 0):
        if ((y % 2) == 1):
            res = (res * x) % self.p
 
        y = math.floor(y/2)
        x = (x * x) % self.p
 
    return res
  
  def __gcd(self, a, b):
    if b == 0:
      return a
    else:
      return self.__gcd(b, a % b)

  
  #Tonelli-Shanks Algorithm
  def __square_root(self, a):

    if self.__gcd(a, self.p) != 1:
      return "a and p not coprime."
    
    if self.__jacobi(a) == -1:
      return 'not available'
    
    q = int(self.p - 1)
    s = 0
    while ((q % 2) == 0):
      q /= 2
      s += 1
    
    z = 2
    while True:
      if self.__power(z, (self.p - 1)/2):
        break
      z += 1
    
    M = s
    c = self.__power(z, q)
    t = self.__power(a, q)
    R = self.__power(a, (q+1)/2)

    while True:
      if t == 0:
        return 0
      elif t == 1:
        return R
      else:
        i = 1
        while i < M:
          if self.__power(t, 2**i) == 1:
            break
          i += 1
        b = self.__power(c, 2**(M-i-1))
        M = i
        c = self.__power(b, 2)
        t = (t * self.__power(b, 2)) % self.p
        R = (R * b) % self.p
  
  def evaluate(self, x):
    if isinstance(x, complex):
      res_x = (self.complex_multiply(self.complex_multiply(x, x), x) + self.complex_multiply(self.A, x) + self.B).real % self.p
      res_y = (self.complex_multiply(self.complex_multiply(x, x), x) + self.complex_multiply(self.A, x) + self.B).imag % self.p
      res = complex(res_x, res_y)
      if not isinstance(self.complex_square_root(res), complex):
        return 'x'
      else:
        return self.complex_square_root(res)
    else:
      res = (x**3 + self.A * x + self.B) % self.p
      if not isinstance(self.__square_root(res), int):
        return 'x'
      else:
        return self.__square_root(res)
  
  def __generate_rand_point(self, state="real"):
    random.seed(5)
    y = 'x'
    if state == "real":
      while not isinstance(y, int):
        x = random.randint(0, self.p)
      # print(x)
      # x = 0
        y = self.evaluate(x)
    
    if state == "complex":
      while not (isinstance(y, int) or isinstance(y, complex)):
        x_real = random.randint(0, self.p)
        x_imag = random.randint(0, self.p)
        x = complex(x_real, x_imag)
      # print(x)
      # x = 0
        y = self.evaluate(x)
    return Point_on_EC(x, y)
  
  def span_check(self, P, S):
    if self.discriminant() == 0:
      raise ValueError("Discriminant of elliptic curve is 0, please change value of 'A' and 'B' in 'y^2 = x^3 + A*x + b'.")

    if (P.x == S.x) and (P.y == S.y):
      return True

    order = self.find_order(P)
    
    Q = P

    for i in range(order):
      Q = self.addition(Q, P)
      if (Q.x == S.x) and (Q.y == S.y):
        return True
    return False

  def distortion_map(self, P):
    if P.x == 'inf':
      x = 0
    else:
      x = -P.x % self.p
    if P.y == 'inf':
      y = 0
    else:
      y = complex(0, P.y % self.p)
    return Point_on_EC(x, y)
  
  def complex_multiply(self, a, b):
    res = a*b
    return complex(res.real % self.p, res.imag % self.p)
  
  def complex_divide(self, a, b):
    left_up = ((a.real * b.real) + (a.imag * b.imag)) % self.p
    left_down = ((b.real * b.real) + (b.imag * b.imag)) % self.p
    right_up = ((a.imag * b.real) - (a.real * b.imag)) % self.p
    right_down = ((b.real * b.real) + (b.imag * b.imag)) % self.p

    if left_down == 0 or right_down == 0:
      raise ValueError("Cannot divide")
    
    # print('lu', left_up)
    # print('ld', left_down)
    # print('ru', right_up)
    # print('rd', right_down)

    if left_up == 0 and right_up == 0:
      return complex(0, 0)
    
    elif left_up == 0:
      left = 0
      right = self.__modulo_divide(right_up, right_down)

    elif right_up == 0:
      left = self.__modulo_divide(left_up, left_down)
      right = 0
    
    else:
      left = self.__modulo_divide(left_up, left_down)
      right = self.__modulo_divide(right_up, right_down)

    # print('l', left)
    # print('r', right)

    return complex(left, right)
  
  def complex_square_root(self, a):
    if a.imag == 0:
      return False

    alpha = int(((a.real*a.real) - ((-1)*a.imag*a.imag)) % self.p)
    gamma = self.__power(alpha, (self.p-1)/2)
    if gamma == -1:
      return False

    alpha = self.__square_root(alpha)
    if not isinstance(alpha, int):
      return False

    delta = self.__modulo_divide((a.real + alpha) % self.p, 2)
    gamma = self.__power(delta, (self.p-1)/2)

    if gamma == -1:
      delta = self.__modulo_divide((a.real-alpha) % self.p, 2)

    real = self.__square_root(delta)
    if not isinstance(real, int):
      return False

    imag = self.__modulo_divide(a.imag, (2*real) % self.p)
    return complex(real, imag)
  
  def gen(self, state):
    return self.__generate_rand_point(state)
    
  def weil_pairing(self, P, Q, S='foo'):
    if self.discriminant() == 0:
      raise ValueError("Discriminant of elliptic curve is 0, please change value of 'A' and 'B' in 'y^2 = x^3 + A*x + b'.")
    
    if isinstance(P, O) or isinstance(Q, O):
      return False
    
    # print('P', P.x, P.y)
    # print('Q', Q.x, Q.y)

    m_p = self.find_order(P)
    m_q = self.find_order(Q)

    same_subgroup = self.span_check(P, Q)
    # print('subgroup', same_subgroup)

    if same_subgroup:
      dist_Q = self.distortion_map(Q)
      # print(dist_Q.x, dist_Q.y)
      m_dist_Q = self.find_order(dist_Q)

    span_p = True
    span_q = True

    if S != 'foo':
      is_span_p = self.span_check(P, S)
      is_span_q = self.span_check(Q, S)

      if is_span_p or is_span_q:
        raise ValueError("Enter point S that is not in the subgroup spanned by P and Q.")
        # print('test1')

      elif not isinstance(S, Point_on_EC):
        raise ValueError("Enter point S on elliptic curve.")
        # print('test2')
      
      elif S.x == S.y == 0:
        pass
      
      elif isinstance(S.x, int) and isinstance(S.y, int):
        if S.y != self.evaluate(S.x) and S.y != -self.evaluate(S.x):
          # print(S.y)
          raise ValueError("S is not a valid point on the elliptic curve.")
          # print('test3')

      elif isinstance(S.x, complex) or isinstance(S.y, complex):
        if S.y != self.evaluate(S.x) and S.y != complex((-self.evaluate(S.x)).real % self.p, (-self.evaluate(S.x)).imag % self.p):
          raise ValueError("S is not a valid point on the elliptic curve.")
    
    else:
      if same_subgroup:
        while span_p or span_q:
          S = self.__generate_rand_point(state="complex")
          span_p = self.span_check(P, S)
          span_q = self.span_check(Q, S)
      
    # print('x', S.x, 'y', S.y)
    # print('px', P.x, 'py', P.y)
    # print('qx', Q.x, 'qy', Q.y)

    min_S = self.subtraction(S)

    if same_subgroup:
      upper_upper = self.__millers_algo(m_p, P, self.addition(dist_Q, S))
      upper_lower = self.__millers_algo(m_p, P, S)

      lower_upper = self.__millers_algo(m_dist_Q, dist_Q, self.addition(P, min_S))
      lower_lower = self.__millers_algo(m_dist_Q, dist_Q, min_S)
    
    else:
      upper_upper = self.__millers_algo(m_p, P, self.addition(Q, S))
      upper_lower = self.__millers_algo(m_p, P, S)

      # print('QS', self.addition(Q, S).x, self.addition(Q, S).y)

      lower_upper = self.__millers_algo(m_q, Q, self.addition(P, min_S))
      lower_lower = self.__millers_algo(m_q, Q, min_S)

      # print('PS', self.addition(P, min_S).x, self.addition(P, min_S).y)
      # print('-S', min_S.x, min_S.y)

    # print('uu', upper_upper)
    # print('ul', upper_lower)
    # print('u', upper)
    # print('lu', lower_upper)
    # print('ll', lower_lower)
    # print('l', lower)

    if same_subgroup:
      upper = self.complex_divide(upper_upper, upper_lower)
      lower = self.complex_divide(lower_upper, lower_lower)
      return self.complex_divide(upper, lower)
    
    else:
      upper = self.__modulo_divide(upper_upper, upper_lower)
      lower = self.__modulo_divide(lower_upper, lower_lower)

    return self.__modulo_divide(upper, lower)
  
  def tate_pairing(self, P, Q, S='foo'):
    if self.discriminant() == 0:
      raise ValueError("Discriminant of elliptic curve is 0, please change value of 'A' and 'B' in 'y^2 = x^3 + A*x + b.")

    if S != 'foo':
      is_span_p = self.span_check(P, S)
      is_span_q = self.span_check(Q, S)

      if is_span_p or is_span_q:
        raise ValueError("Enter point S that is not in the subgroup spanned by P and Q.")
        # print('test1')

      elif not isinstance(S, Point_on_EC):
        raise ValueError("Enter point S on elliptic curve.")
        # print('test2')
      
      elif S.y != self.evaluate(S.x) and S.y != (-self.evaluate(S.x)) % self.p:
        raise ValueError("S is not a valid point on the elliptic curve.")
        # print('test3')
    
    else:
      m_p = self.find_order(P)
      span_p = True
      while span_p:
        S = self.__generate_rand_point()
        span_p = self.span_check(P, S)
    
    upper = self.__millers_algo(m_p, P, self.addition(Q, S))
    lower = self.__millers_algo(m_p, P, S)
  
    return self.__modulo_divide(upper, lower)

In [44]:
# defining small generator and simple bilinear mapping
p = 137
gx = 67
modular = 547
polynom = [[1, 0, 1, 5], 35]
# polynom = [[1, 0, 1, 6], 36]
u = 3
A = 1
B = 0
E = Elliptic_curve(A, B, modular)
g = Point_on_EC(gx, E.evaluate(gx))

In [45]:
print(g.x, g.y)

67 66


In [46]:
print(E.multiply(g, p))

In [47]:
# for i in range(1, modular):
#   try:
#     E.evaluate(i)
#   except ValueError:
#     print(i, 'e')
#   else:
#     xxx = Point_on_EC(i, E.evaluate(i))
#     print('x:', xxx.x, 'y:', xxx.y)
#     print('n:', E.find_order(xxx))

In [48]:
# for i in range(p+1):
#   print(i,':', E.multiply(g, i).x, E.multiply(g, i).y)

## QAP

### QAP Insides

In [49]:
# This part is just the QAP
# Reference taken from https://medium.com/@VitalikButerin/quadratic-arithmetic-programs-from-zero-to-hero-f6d558cea649

# Here the input_array is coefficients of the polynomials,
# first part of the list is the coefficients of the left polynomials,
# the right part is the coeffiecients of the right polynomials.

# The polynomals used in here is x**3+x+5=35
# d is the degree of polynomials on the left side
# u is the solution to the polynomials
# y is the target value
# v is the variables vector
# threshold is the limit, if a value is below this threshold then it's considered 0 (only NOT in moduloz)
# input_array = [[1, 0, 1, 5], [35]]
# d = len(input_array[0])-1
# u = 3
# y = input_array[-1][0] % p
# threshold = 10**10

def coeff_builder(u):
  v = [1, u]
  d = len(polynom[0])-1
  # a to c values here is the placeholder for the left(a), right(b), and output values(c),
  # which will be turned into 1 and 0 vectors later (with the exception of the addition of a constant)
  a_values = []
  b_values = []
  c_values = []

  # Defining the appending function to append the v vector as it should not have same values in it
  def appending(li, value):
    if value not in li and value!=0:
      li.append(value)

  # Now we want to build the v vector while also build the a,b,c, values
  initial_sum = u
  # Total here is the place holder of each of the elements in the polynomials
  # [ax**d, bx**(d-1), ..., c]
  total = [0]

  # Here we iterate from degree 1 until d, notice we didn't include the constant, as it would be added at the end
  for i in range(d):
    # Skip if coefficient is 0
    if polynom[0][i] == 0:
      continue

    # If the coefficient of an element is not 1 or 0, we need to add them to a, b, c, because it is another operation
    if polynom[0][i] != 1 and polynom[0][i] != 0:
      a_values.append([polynom[0][i]])
      b_values.append([u])
      appending(v, polynom[0][i])
      c_values.append([(polynom[0][i]*u) % p])

    sums = (initial_sum * polynom[0][i]) % p
    appending(v, sums)

    # Here we calculate a*x**(i)
    for j in range(d-i-1):
      a_values.append([sums])
      b_values.append([u])
      sums = (sums * u) % p
      c_values.append([sums])
      appending(v, sums)

    # Appending the next element to total
    total.append(sums)

    # Here we add the elements one by one
    if i != 0:
      a_values.append([sum(i for i in total[:-1]), total[-1]])
      b_values.append([1])
      c_values.append([sum(i for i in total)])
  
    appending(v, sum(i for i in total))

  # The last one is for the constant
  if polynom[0][-1] != 0:
    a_values.append([sum(i for i in total) % p])
    a_values[-1].append(polynom[0][-1])
    b_values.append([1])
    c_values.append([(sum(a_values[-1])*b_values[-1][0]) % p])
    v.insert(2, c_values[-1][0])

  return v, a_values, b_values, c_values

v, l_values, r_values, o_values = coeff_builder(u)

def coeff_to_c1rs(li, variables):
  result = []
  for i in li:
    z = [1 if j in i else 0 for j in variables]
    result.append(z)
  return result

l = coeff_to_c1rs(l_values, v)
r = coeff_to_c1rs(r_values, v)
o = coeff_to_c1rs(o_values, v)
if polynom[0][-1] != 0:
  l[-1][0] = polynom[0][-1]
  if polynom[0][-1] in v:
    l[-1][v.index(polynom[0][-1])] = 0

print('v')
print(v)
print('l')
print(l)
print('r')
print(r)
print('o')
print(o)

v
[1, 3, 35, 9, 27, 30]
l
[[0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 1, 0], [5, 0, 0, 0, 0, 1]]
r
[[0, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0]]
o
[[0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1], [0, 0, 1, 0, 0, 0]]


In [50]:
def polynom_multi(a, b, p):
  result = [0 for i in range(len(a)+len(b)-1)]
  for i in range(len(a)):
    for j in range(len(b)):
      result[i+j] = (result[i+j] + (a[i]*b[j])) % p
  return result

In [51]:
def modulo_divide(p, q, mod):
  for i in range(1, mod):
    if (i * q) % mod == p:
      return i

In [52]:
def lagrange_interpolation(y, p):
  polynom_coeff = []
  for i in range(len(y[0])):
    total = []
    another_placeholder = []
    another_placeholder.append([0 for i in range(len(y))])
    for j in range(len(y)):
      denominator = 1
      if y[j][i] == 0:
        continue
      placeholder = []
      for k in range(len(y)):
        if k == j:
          continue
        placeholder.append([1, -k-1])
        denominator = (denominator * (j-k)) % p

      total = [((modulo_divide(1, denominator, p)) * y[j][i]) % p]
      
      for pair in placeholder:
        total = polynom_multi(pair, total, p)

      another_placeholder.append(total)
    
    result = [0 for i in range(len(another_placeholder[0]))]

    if len(another_placeholder) > 1:
      for value in another_placeholder:
        for k in range(len(value)):
          result[k] = (result[k] + value[k]) % p

    polynom_coeff.append(result)

  return polynom_coeff

In [53]:
l_polynom_coeff = lagrange_interpolation(l, p)
r_polynom_coeff = lagrange_interpolation(r, p)
o_polynom_coeff = lagrange_interpolation(o, p)

$$a_1(x)=10x^3+6x^2+0x+6$$
$$a_2(x)=3x^3+5x^2+7x+6$$
$$a_3(x)=0x^3+0x^2+0x+0$$
$$a_4(x)=6x^3+7x^2+4x+5$$
$$a_5(x)=5x^3+9x^2+4x+4$$
$$a_6(x)=2x^3+10x^2+0x+10$$

In [54]:
l_polynom_coeff

[[115, 132, 32, 132],
 [45, 5, 80, 8],
 [0, 0, 0, 0],
 [69, 133, 78, 131],
 [68, 72, 130, 4],
 [23, 136, 116, 136]]

$$b_1(x)=7x^3+8x^2+4x+3$$
$$b_2(x)=4x^3+5x^2+7x+9$$
$$b_3(x)=0x^3+0x^2+0x+0$$
$$b_4(x)=0x^3+0x^2+0x+0$$
$$b_5(x)=0x^3+0x^2+0x+0$$
$$b_6(x)=0x^3+0x^2+0x+0$$

In [55]:
r_polynom_coeff

[[91, 71, 109, 3],
 [46, 66, 28, 135],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0]]

$$c_1(x)=0x^3+0x^2+0x+0$$
$$c_2(x)=0x^3+0x^2+0x+0$$
$$c_3(x)=2x^3+10x^2+0x+10$$
$$c_4(x)=9x^3+7x^2+3x+4$$
$$c_5(x)=6x^3+7x^2+4x+5$$
$$c_6(x)=5x^3+9x^2+4x+4$$

In [56]:
o_polynom_coeff

[[0, 0, 0, 0],
 [0, 0, 0, 0],
 [23, 136, 116, 136],
 [114, 70, 87, 4],
 [69, 133, 78, 131],
 [68, 72, 130, 4]]

### QAP Setup

In [57]:
def setup_QAP(key_status):
  v, l_values, r_values, o_values = coeff_builder(u)

  l = coeff_to_c1rs(l_values, v)
  r = coeff_to_c1rs(r_values, v)
  o = coeff_to_c1rs(o_values, v)
  
  if polynom[0][-1] != 0:
    l[-1][0] = polynom[0][-1]
    if polynom[0][-1] in v:
      l[-1][v.index(polynom[0][-1])] = 0
  
  l_polynom_coeff = lagrange_interpolation(l, p)
  r_polynom_coeff = lagrange_interpolation(r, p)
  o_polynom_coeff = lagrange_interpolation(o, p)

  z = []
  for i in range(len(l)):
    z.append([1, -i-1])
  t = [1]
  for i in z:
    t = polynom_multi(t, i, p)
  
  if key_status == "p":
    return l_polynom_coeff[3:], r_polynom_coeff[3:], o_polynom_coeff[3:], t
  elif key_status == "v":
    return l_polynom_coeff[0:3], r_polynom_coeff[0:3], o_polynom_coeff[0:3], t
  elif key_status == "a":
    return l_polynom_coeff, r_polynom_coeff, o_polynom_coeff, t
  else:
    return "Please, specified key_status to 'p' for proving or 'v' for verification"

setup_QAP('p')

([[69, 133, 78, 131], [68, 72, 130, 4], [23, 136, 116, 136]],
 [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]],
 [[114, 70, 87, 4], [69, 133, 78, 131], [68, 72, 130, 4]],
 [1, 127, 35, 87, 24])

## Sample Random

In [58]:
# sample random
import random

def sample_random():
  random.seed(17)
  s = random.randint(1, p-1)
  rho_l = random.randint(1, p-1)
  rho_r = random.randint(1, p-1)
  alpha_l = random.randint(1, p-1)
  alpha_r = random.randint(1, p-1)
  alpha_o = random.randint(1, p-1)
  beta = random.randint(1, p-1)
  gamma = random.randint(1, p-1)

  rho_o = (rho_l*rho_r) % p
  gl = E.multiply(g, rho_l)
  gr = E.multiply(g, rho_r)
  go = E.multiply(g, rho_o)
  
  return s, alpha_l, alpha_r, alpha_o, beta, gamma, gl, gr, go
  # return s, rho_l, rho_r, rho_o, alpha_l, alpha_r, alpha_o, beta, gamma, gl.x, gl.y, gr.x, gr.y, go.x, go.y

sample_random()

(134,
 94,
 75,
 45,
 72,
 29,
 <__main__.Point_on_EC at 0x7fd796007750>)

## Proving Key

In [59]:
# set the proving key
# random_values = sample_random(p, modular)
def proving_key_setup():

  left, right, output, t = setup_QAP('p')
  s, alpha_l, alpha_r, alpha_o, beta, gamma, gl, gr, go = sample_random()

  proving_key = {}
  zero_knowledge = {}

  degree = len(left[0])
  # print(degree)
  evaluation = []
  
  for i in range(2*degree):
    evaluation.append(E.multiply(g, s**i))
  evaluation.reverse()

  prover_l = []
  prover_r = []
  prover_o = []
  prover_alpha_l = []
  prover_alpha_r = []
  prover_alpha_o = []
  beta_polynomials = []
  for i in range(len(left)):
    prover_l_placeholder = O()
    prover_r_placeholder = O()
    prover_o_placeholder = O()
    alpha_r_placeholder = O()
    alpha_l_placeholder = O()
    alpha_o_placeholder = O()
    beta_l_placeholder = O()
    beta_r_placeholder = O()
    beta_o_placeholder = O()
    for j in range(len(left[0])):
      prover_l_placeholder = E.addition(prover_l_placeholder, E.multiply(E.multiply(gl, s**(degree-(j+1))), left[i][j]))
      # prover_l_placeholder = (prover_l_placeholder + ((((gl**((s**(degree-(j+1))) % p)) % modular) ** left[i][j]) % modular)) % modular
      prover_r_placeholder = E.addition(prover_r_placeholder, E.multiply(E.multiply(gr, s**(degree-(j+1))), right[i][j]))
      prover_o_placeholder = E.addition(prover_o_placeholder, E.multiply(E.multiply(go, s**(degree-(j+1))), output[i][j]))

      alpha_l_placeholder = E.addition(alpha_l_placeholder, E.multiply(E.multiply(gl, alpha_l*(s**(degree-(j+1)))), left[i][j]))
      alpha_r_placeholder = E.addition(alpha_r_placeholder, E.multiply(E.multiply(gr, alpha_r*(s**(degree-(j+1)))), right[i][j]))
      alpha_o_placeholder = E.addition(alpha_o_placeholder, E.multiply(E.multiply(go, alpha_o*(s**(degree-(j+1)))), output[i][j]))
      # alpha_l_placeholder = (alpha_l_placeholder * ((((gl**(alpha_l*((s**(degree-(j+1))) % p) % p)) % modular) ** left[i][j]) % modular)) % modular
      # alpha_r_placeholder = (alpha_r_placeholder * ((((gr**(alpha_r*((s**(degree-(j+1))) % p) % p)) % modular) ** right[i][j]) % modular)) % modular
      # alpha_o_placeholder = (alpha_o_placeholder * ((((go**(alpha_o*((s**(degree-(j+1))) % p) % p)) % modular) ** output[i][j]) % modular)) % modular

      beta_l_placeholder = E.addition(beta_l_placeholder, E.multiply(E.multiply(gl, beta*(s**(degree-(j+1)))), left[i][j])) 
      beta_r_placeholder = E.addition(beta_r_placeholder, E.multiply(E.multiply(gr, beta*(s**(degree-(j+1)))), right[i][j])) 
      beta_o_placeholder = E.addition(beta_o_placeholder, E.multiply(E.multiply(go, beta*(s**(degree-(j+1)))), output[i][j])) 
      # beta_l_placeholder = (beta_l_placeholder * ((((gl**(beta*((s**(degree-(j+1))) % p) % p)) % modular) ** left[i][j]) % modular)) % modular
      # beta_r_placeholder = (beta_r_placeholder * ((((gr**(beta*((s**(degree-(j+1))) % p) % p)) % modular) ** right[i][j]) % modular)) % modular
      # beta_o_placeholder = (beta_o_placeholder * ((((go**(beta*((s**(degree-(j+1))) % p) % p)) % modular) ** output[i][j]) % modular)) % modular
      beta_placeholder = E.addition(E.addition(beta_l_placeholder, beta_r_placeholder), beta_o_placeholder)
      # beta_placeholder = (beta_l_placeholder * beta_r_placeholder * beta_o_placeholder) % modular
    prover_l.append(prover_l_placeholder)
    prover_r.append(prover_r_placeholder)
    prover_o.append(prover_o_placeholder)
    prover_alpha_l.append(alpha_l_placeholder)
    prover_alpha_r.append(alpha_r_placeholder)
    prover_alpha_o.append(alpha_o_placeholder)
    beta_polynomials.append(beta_placeholder)

  proving_key["evaluation"] = evaluation

  proving_key["prover_left_polynomials"] = prover_l
  proving_key["prover_right_polynomials"] = prover_r
  proving_key["prover_output_polynomials"] = prover_o

  proving_key["prover_shifted_left_polynomials"] = prover_alpha_l
  proving_key["prover_shifted_right_polynomials"] = prover_alpha_r
  proving_key["prover_shifted_output_polynomials"] = prover_alpha_o

  proving_key["beta_polynomials"] = beta_polynomials

  # zero knowledge part
  left_target_polynomial = O()
  right_target_polynomial = O()
  output_target_polynomial = O()

  shifted_left_target_polynomial = O()
  shifted_right_target_polynomial = O()
  shifted_output_target_polynomial = O()

  beta_left_target_polynomial = O()
  beta_right_target_polynomial = O()
  beta_output_target_polynomial = O()

  for i in range(len(t)):
    left_target_polynomial = E.addition(left_target_polynomial, E.multiply(E.multiply(gl, s**(len(t)-(i+1))), t[i]))
    right_target_polynomial = E.addition(right_target_polynomial, E.multiply(E.multiply(gr, s**(len(t)-(i+1))), t[i]))
    output_target_polynomial = E.addition(output_target_polynomial, E.multiply(E.multiply(go, s**(len(t)-(i+1))), t[i]))
    # left_target_polynomial = (left_target_polynomial * ((((gl**((s**(len(t)-(i+1))) % p)) % modular) ** t[i]) % modular)) % modular
    # right_target_polynomial = (right_target_polynomial * ((((gr**((s**(len(t)-(i+1))) % p)) % modular) ** t[i]) % modular)) % modular
    # output_target_polynomial = (output_target_polynomial * ((((go**((s**(len(t)-(i+1))) % p)) % modular) ** t[i]) % modular)) % modular

    shifted_left_target_polynomial = E.addition(shifted_left_target_polynomial, E.multiply(E.multiply(gl, alpha_l*(s**(len(t)-(i+1)))), t[i]))
    shifted_right_target_polynomial = E.addition(shifted_right_target_polynomial, E.multiply(E.multiply(gr, alpha_r*(s**(len(t)-(i+1)))), t[i]))
    shifted_output_target_polynomial = E.addition(shifted_output_target_polynomial, E.multiply(E.multiply(go, alpha_o*(s**(len(t)-(i+1)))), t[i]))
    # shifted_left_target_polynomial = (shifted_left_target_polynomial * ((((gl**(alpha_l*((s**(len(t)-(i+1))) % p) % p)) % modular) ** t[i]) % modular)) % modular
    # shifted_right_target_polynomial = (shifted_right_target_polynomial * ((((gr**(alpha_r*((s**(len(t)-(i+1))) % p) % p)) % modular) ** t[i]) % modular)) % modular
    # shifted_output_target_polynomial = (shifted_output_target_polynomial * ((((go**(alpha_o*((s**(len(t)-(i+1))) % p) % p)) % modular) ** t[i]) % modular)) % modular

    beta_left_target_polynomial = E.addition(beta_left_target_polynomial, E.multiply(E.multiply(gl, beta*(s**(len(t)-(i+1)))), t[i]))
    beta_right_target_polynomial = E.addition(beta_right_target_polynomial, E.multiply(E.multiply(gr, beta*(s**(len(t)-(i+1)))), t[i]))
    beta_output_target_polynomial = E.addition(beta_output_target_polynomial, E.multiply(E.multiply(go, beta*(s**(len(t)-(i+1)))), t[i]))
    # beta_left_target_polynomial = (beta_left_target_polynomial * ((((gl**(beta*((s**(len(t)-(i+1))) % p) % p)) % modular) ** t[i]) % modular)) % modular
    # beta_right_target_polynomial = (beta_right_target_polynomial * ((((gr**(beta*((s**(len(t)-(i+1))) % p) % p)) % modular) ** t[i]) % modular)) % modular
    # beta_output_target_polynomial = (beta_output_target_polynomial * ((((go**(beta*((s**(len(t)-(i+1))) % p) % p)) % modular) ** t[i]) % modular)) % modular
  
  zero_knowledge["left_target_polynomial"] = left_target_polynomial
  zero_knowledge["right_target_polynomial"] = right_target_polynomial
  zero_knowledge["output_target_polynomial"] = output_target_polynomial

  zero_knowledge["shifted_left_target_polynomial"] = shifted_left_target_polynomial
  zero_knowledge["shifted_right_target_polynomial"] = shifted_right_target_polynomial
  zero_knowledge["shifted_output_target_polynomial"] = shifted_output_target_polynomial

  zero_knowledge["beta_left_target_polynomial"] = beta_left_target_polynomial
  zero_knowledge["beta_right_target_polynomial"] = beta_right_target_polynomial
  zero_knowledge["beta_output_target_polynomial"] = beta_output_target_polynomial

  proving_key["zero_knowledge"] = zero_knowledge

  # proving_key.append(evaluation)
  # proving_key.append([prover_l, prover_r, prover_o])
  # proving_key.append([prover_alpha_l, prover_alpha_r, prover_alpha_o])
  # proving_key.append([beta_l, beta_r, beta_o])
  return proving_key

In [60]:
proving_key = proving_key_setup()
proving_key

{'beta_polynomials': [<__main__.Point_on_EC at 0x7fd79600c1d0>,
 'evaluation': [<__main__.Point_on_EC at 0x7fd7961845d0>,
 'prover_left_polynomials': [<__main__.Point_on_EC at 0x7fd79600ced0>,
 'prover_output_polynomials': [<__main__.Point_on_EC at 0x7fd79600cb90>,
 'prover_right_polynomials': [<__main__.O at 0x7fd79600c810>,
 'prover_shifted_left_polynomials': [<__main__.Point_on_EC at 0x7fd79600cd90>,
 'prover_shifted_output_polynomials': [<__main__.Point_on_EC at 0x7fd79618a390>,
 'prover_shifted_right_polynomials': [<__main__.O at 0x7fd79600c190>,
 'zero_knowledge': {'beta_left_target_polynomial': <__main__.Point_on_EC at 0x7fd79600c750>,
  'beta_output_target_polynomial': <__main__.Point_on_EC at 0x7fd79600cbd0>,
  'beta_right_target_polynomial': <__main__.Point_on_EC at 0x7fd79600c910>,
  'left_target_polynomial': <__main__.Point_on_EC at 0x7fd79600c390>,
  'output_target_polynomial': <__main__.Point_on_EC at 0x7fd79600c350>,
  'right_target_polynomial': <__main__.Point_on_EC at 

## Verification Key

In [61]:
#set the verification key
def verification_key_setup():
  left, right, output, t = setup_QAP('v')
  s, alpha_l, alpha_r, alpha_o, beta, gamma, gl, gr, go = sample_random()
  verification_key = {}

  degree = len(left[0])

  g_1 = g

  go_t = O()
  for i in range(len(t)):
    go_t = E.addition(go_t, E.multiply(E.multiply(go, s ** (len(t)-(i+1))), t[i]))
    # go_t = (go_t * ((((go ** ((s ** (len(t)-(i+1))) % p)) % modular) ** t[i]) % modular)) % modular
    
  ver_l = []
  ver_r = []
  ver_o = []
  for i in range(3):
    ver_l_placeholder = O()
    ver_r_placeholder = O()
    ver_o_placeholder = O()
    for j in range(len(left[0])):
      ver_l_placeholder = E.addition(ver_l_placeholder, E.multiply(E.multiply(gl, s**(degree-(j+1))), left[i][j]))
      ver_r_placeholder = E.addition(ver_r_placeholder, E.multiply(E.multiply(gr, s**(degree-(j+1))), right[i][j]))
      ver_o_placeholder = E.addition(ver_o_placeholder, E.multiply(E.multiply(go, s**(degree-(j+1))), output[i][j]))
      # ver_l_placeholder = (ver_l_placeholder * ((((gl**((s**(degree-(j+1))) % p)) % modular) ** left[i][j]) % modular)) % modular
      # ver_r_placeholder = (ver_r_placeholder * ((((gr**((s**(degree-(j+1))) % p)) % modular) ** right[i][j]) % modular)) % modular
      # ver_o_placeholder = (ver_o_placeholder * ((((go**((s**(degree-(j+1))) % p)) % modular) ** output[i][j]) % modular)) % modular
    ver_l.append(ver_l_placeholder)
    ver_r.append(ver_r_placeholder)
    ver_o.append(ver_o_placeholder)

  g_alpha_l = E.multiply(g, alpha_l)
  g_alpha_r = E.multiply(g, alpha_r)
  g_alpha_o = E.multiply(g, alpha_o)

  g_gamma = E.multiply(g, gamma)
  g_beta_gamma = E.multiply(g, beta*gamma)

  verification_key["generator"] = g_1
  verification_key["target_polynomials"] = go_t

  verification_key["verifier_left_polynomials"] = ver_l
  verification_key["verifier_right_polynomials"] = ver_r
  verification_key["verifier_output_polynomials"] = ver_o

  verification_key["alpha_left"] = g_alpha_l
  verification_key["alpha_right"] = g_alpha_r
  verification_key["alpha_output"] = g_alpha_o

  verification_key["gamma"] = g_gamma
  verification_key["beta_gamma"] = g_beta_gamma

  # verification_key.append(g_1)
  # verification_key.append(go_t)
  # verification_key.append(ver_l)
  # verification_key.append(ver_r)
  # verification_key.append(ver_o)
  # verification_key.append(g_alpha_l)
  # verification_key.append(g_alpha_r)
  # verification_key.append(g_alpha_o)
  # verification_key.append(g_gamma)
  # verification_key.append(g_beta_gamma)
  return verification_key

In [62]:
verification_key = verification_key_setup()
verification_key

{'alpha_left': <__main__.Point_on_EC at 0x7fd7960159d0>,
 'alpha_output': <__main__.Point_on_EC at 0x7fd796015750>,
 'alpha_right': <__main__.Point_on_EC at 0x7fd796015450>,
 'beta_gamma': <__main__.Point_on_EC at 0x7fd796007d10>,
 'gamma': <__main__.Point_on_EC at 0x7fd7960077d0>,
 'generator': <__main__.Point_on_EC at 0x7fd7993bdfd0>,
 'target_polynomials': <__main__.Point_on_EC at 0x7fd796015390>,
 'verifier_left_polynomials': [<__main__.Point_on_EC at 0x7fd79b44c690>,
 'verifier_output_polynomials': [<__main__.O at 0x7fd7960158d0>,
 'verifier_right_polynomials': [<__main__.Point_on_EC at 0x7fd796015d10>,
  <__main__.O at 0x7fd796015c10>]}

## Setup

In [63]:
def setup():
  random_values = sample_random()
  proving_key = proving_key_setup()
  verification_key = verification_key_setup()

  return proving_key, verification_key

In [64]:
proving_key, verification_key = setup()

In [65]:
proving_key

{'beta_polynomials': [<__main__.Point_on_EC at 0x7fd796110250>,
 'evaluation': [<__main__.Point_on_EC at 0x7fd796007f10>,
 'prover_left_polynomials': [<__main__.Point_on_EC at 0x7fd796110210>,
 'prover_output_polynomials': [<__main__.Point_on_EC at 0x7fd796110710>,
 'prover_right_polynomials': [<__main__.O at 0x7fd796110390>,
 'prover_shifted_left_polynomials': [<__main__.Point_on_EC at 0x7fd796001690>,
 'prover_shifted_output_polynomials': [<__main__.Point_on_EC at 0x7fd79610d1d0>,
 'prover_shifted_right_polynomials': [<__main__.O at 0x7fd796110ad0>,
 'zero_knowledge': {'beta_left_target_polynomial': <__main__.Point_on_EC at 0x7fd79618aed0>,
  'beta_output_target_polynomial': <__main__.Point_on_EC at 0x7fd7961812d0>,
  'beta_right_target_polynomial': <__main__.Point_on_EC at 0x7fd79618a190>,
  'left_target_polynomial': <__main__.Point_on_EC at 0x7fd79600c210>,
  'output_target_polynomial': <__main__.Point_on_EC at 0x7fd79600c990>,
  'right_target_polynomial': <__main__.Point_on_EC at 

In [66]:
verification_key

{'alpha_left': <__main__.Point_on_EC at 0x7fd7960fb3d0>,
 'alpha_output': <__main__.Point_on_EC at 0x7fd7960fbd10>,
 'alpha_right': <__main__.Point_on_EC at 0x7fd7960fb1d0>,
 'beta_gamma': <__main__.Point_on_EC at 0x7fd796010110>,
 'gamma': <__main__.Point_on_EC at 0x7fd796010150>,
 'generator': <__main__.Point_on_EC at 0x7fd7993bdfd0>,
 'target_polynomials': <__main__.Point_on_EC at 0x7fd796110a10>,
 'verifier_left_polynomials': [<__main__.Point_on_EC at 0x7fd796181c90>,
 'verifier_output_polynomials': [<__main__.O at 0x7fd796110350>,
 'verifier_right_polynomials': [<__main__.Point_on_EC at 0x7fd796110dd0>,
  <__main__.O at 0x7fd7960fbe50>]}

# Proving

In [67]:
def variables_values(witness):
  v, l, r, o = coeff_builder(witness)
  return v

In [68]:
witness = 3
v = variables_values(witness)
v

[1, 3, 35, 9, 27, 30]

In [69]:
def normalize(poly):
    while poly and poly[0] == 0:
        poly.pop(0)
    return poly
    
def polynom_div(p1, p2, mod):
  diff = len(p1)-len(p2)
  result = [0]*len(p1)

  if diff < 0 or all(i == 0 for i in p2):
    return False

  for i in range(diff+1):
    if len(p1)<len(p2):
      break

    if p1[i] != 0:
      a = modulo_divide(p1[i], p2[0], mod)
      result[len(p1)+i-(diff+1)] = a

      ph = [*[0]*i, *result[len(p1)+i-(diff+1):]]
      zipped = zip(p1, polynom_multi(p2, ph, mod))
      j = 0
      for a,b in zipped:
        p1[j] = (a-b) % mod
        j += 1

  remainder = p1
  return normalize(result), normalize(remainder)

# h, rem = polynom_div(px, t, p)

In [70]:
# h

In [71]:
# rem

In [72]:
def check_QAP(polynom_coeff, variables, p):
  result = [0 for i in range(len(polynom_coeff[0]))]
  for i in range(len(polynom_coeff)):
    for j in range(len(polynom_coeff[i])):
      result[j] = (result[j] + (variables[i] * polynom_coeff[i][j])) % p
  return result

In [73]:
# L = [0 for i in range(len(l_polynom_coeff[0]))]
# for i in range(len(l_polynom_coeff)):
#   for j in range(len(l_polynom_coeff[i])):
#     L[j] = (L[j] + (v[i] * l_polynom_coeff[i][j])) % p
# L

In [74]:
# R = [0 for i in range(len(r_polynom_coeff[0]))]
# for i in range(len(r_polynom_coeff)):
#   for j in range(len(r_polynom_coeff[i])):
#     R[j] = (R[j] + (v[i] * r_polynom_coeff[i][j])) % p
# R

In [75]:
# O = [0 for i in range(len(o_polynom_coeff[0]))]
# for i in range(len(o_polynom_coeff)):
#   for j in range(len(o_polynom_coeff[i])):
#     O[j] = (O[j] + (v[i] * o_polynom_coeff[i][j])) % p
# O

In [76]:
# finding t(x) which is the target polynomial (A(x) * B(x) - C(x))
# L_and_R = polynom_multi(L,R,p)
# lr_o_diff = len(L_and_R) - len(O)
# O = [0]*lr_o_diff + O
# px = [0 for i in range(len(O))]
# for i in range(len(O)):
#   px[i] = (L_and_R[i] - O[i]) % p
# print(L_and_R)
# print(O)
# print(px)

In [77]:
def prover_sample_random():
  random.seed(17)
  delta_l = random.randint(0, p-1)
  delta_r = random.randint(0, p-1)
  delta_o = random.randint(0, p-1)

  return delta_l, delta_r, delta_o
# a, b, c = prover_sample_random()
# print(a)
# print(b)
# print(c)

In [78]:
def polynom_add_sub(a, b, operation, p):
  if not isinstance(a, list) or not isinstance(b, list):
    return "Input two polynomials"
  
  diff = len(a)-len(b)

  if diff < 0:
    a = [0]*abs(diff) + a
  elif diff > 0:
    b = [0]*abs(diff) + b
  
  result = [0 for i in range(len(a))]

  for i in range(len(a)):
    if operation == "add":
      # print('i', i)
      result[i] = (a[i] + b[i]) % p
      # print(result)
    elif operation == "sub":
      # print('i', i)
      result[i] = (a[i] - b[i]) % p
      # print(result)
  
  return result

In [79]:
# polynom_add_sub([1, 2, 3], [2], "sub", p)

In [80]:
def QAP_witness(witness):
  l_values, r_values, o_values, t = setup_QAP('a')
  prover_v = variables_values(witness)
  valid_v = variables_values(u)
  delta_l, delta_r, delta_o = prover_sample_random()
  print('p', prover_v)
  print('v', valid_v)
  # v, l_values, r_values, o_values = coeff_builder(input_array, u, p)

  # l = coeff_to_c1rs(l_values, v)
  # r = coeff_to_c1rs(r_values, v)
  # o = coeff_to_c1rs(o_values, v)
  # if input_array[0][-1] != 0:
  #   l[-1][0] = input_array[0][-1]
  #   if input_array[0][-1] in v:
  #     l[-1][v.index(input_array[0][-1])] = 0
  
  # prover_l_polynom_coeff = lagrange_interpolation(l, p)
  # prover_r_polynom_coeff = lagrange_interpolation(r, p)
  # prover_o_polynom_coeff = lagrange_interpolation(o, p)

  prover_l = check_QAP(l_values, prover_v, p)
  prover_r = check_QAP(r_values, prover_v, p)
  prover_o = check_QAP(o_values, prover_v, p)
  
  z = []
  for i in range(len(prover_l)):
    z.append([1, -i-1])
  t = [1]
  for i in z:
    t = polynom_multi(t, i, p)
  
  L_and_R = polynom_multi(prover_l,prover_r,p)
  px = polynom_add_sub(L_and_R, prover_o, "sub", p)
  # print('lr',L_and_R)
  # print('lro',px)

  h, rem = polynom_div(px, t, p)

  h = polynom_add_sub(h, polynom_multi([delta_r], prover_l, p), "add", p)
  h = polynom_add_sub(h, polynom_multi([delta_l], prover_r, p), "add", p)
  h = polynom_add_sub(h, polynom_multi([((delta_l * delta_r) % p)], t, p), "add", p)

  h[-1] = (h[-1] - delta_o) % p

  return h, prover_v[3:]

In [81]:
# a, b = QAP_witness(0)

In [82]:
#  a

In [83]:
# b

In [84]:
# h_x, prover_v = QAP_witness(witness)

In [85]:
# h_x

In [86]:
# prover_v

In [87]:
def apply_values(values, encrypted):
  result = O()
  for i in range(len(values)):
    result = E.addition(result, E.multiply(encrypted[i], values[i]))
    # result = (result * ((encrypted[i] ** values[i]) % modular)) % modular
  return result

In [88]:
def prover_proof(proving_key, witness):
  if witness >= p:
    return False
  proof = {}

  h, prover_values= QAP_witness(witness)
  delta_l, delta_r, delta_o = prover_sample_random()
  # print(h)
  # print(prover_values)

  prover_left_polynomial = E.addition(E.multiply(proving_key["zero_knowledge"]["left_target_polynomial"], delta_l), apply_values(prover_values, proving_key["prover_left_polynomials"]))
  prover_right_polynomial = E.addition(E.multiply(proving_key["zero_knowledge"]["right_target_polynomial"], delta_r), apply_values(prover_values, proving_key["prover_right_polynomials"]))
  prover_output_polynomial = E.addition(E.multiply(proving_key["zero_knowledge"]["output_target_polynomial"], delta_o), apply_values(prover_values, proving_key["prover_output_polynomials"]))
  # prover_left_polynomial = (((proving_key["zero_knowledge"]["left_target_polynomial"] ** delta_l) % modular) * apply_values(prover_values, proving_key["prover_left_polynomials"])) % modular
  # prover_right_polynomial = (((proving_key["zero_knowledge"]["right_target_polynomial"] ** delta_r) % modular) * apply_values(prover_values, proving_key["prover_right_polynomials"])) % modular
  # prover_output_polynomial = (((proving_key["zero_knowledge"]["output_target_polynomial"] ** delta_o) % modular) * apply_values(prover_values, proving_key["prover_output_polynomials"])) % modular

  prover_shifted_left_polynomial = E.addition(E.multiply(proving_key["zero_knowledge"]["shifted_left_target_polynomial"], delta_l), apply_values(prover_values, proving_key["prover_shifted_left_polynomials"]))
  prover_shifted_right_polynomial = E.addition(E.multiply(proving_key["zero_knowledge"]["shifted_right_target_polynomial"], delta_r), apply_values(prover_values, proving_key["prover_shifted_right_polynomials"]))
  prover_shifted_output_polynomial = E.addition(E.multiply(proving_key["zero_knowledge"]["shifted_output_target_polynomial"], delta_o), apply_values(prover_values, proving_key["prover_shifted_output_polynomials"]))
  # prover_shifted_left_polynomial = (((proving_key["zero_knowledge"]["shifted_left_target_polynomial"] ** delta_l) % modular) * apply_values(prover_values, proving_key["prover_shifted_left_polynomials"])) % modular
  # prover_shifted_right_polynomial = (((proving_key["zero_knowledge"]["shifted_right_target_polynomial"] ** delta_r) % modular) * apply_values(prover_values, proving_key["prover_shifted_right_polynomials"])) % modular
  # prover_shifted_output_polynomial = (((proving_key["zero_knowledge"]["shifted_output_target_polynomial"] ** delta_o) % modular) * apply_values(prover_values, proving_key["prover_shifted_output_polynomials"])) % modular

  prover_result_polynomial = apply_values(h, proving_key["evaluation"][-len(h):])
  
  prover_consistency_check_polynomial_1 = E.multiply(proving_key["zero_knowledge"]["beta_left_target_polynomial"], delta_l)
  prover_consistency_check_polynomial_2 = E.multiply(proving_key["zero_knowledge"]["beta_right_target_polynomial"], delta_r)
  prover_consistency_check_polynomial_3 = E.multiply(proving_key["zero_knowledge"]["beta_output_target_polynomial"], delta_o)
  prover_consistency_check_polynomial = E.addition(E.addition(E.addition(prover_consistency_check_polynomial_1, prover_consistency_check_polynomial_2), prover_consistency_check_polynomial_3), apply_values(prover_values, proving_key["beta_polynomials"]))
  # prover_consistency_check_polynomial = (((proving_key["zero_knowledge"]["beta_output_target_polynomial"] ** delta_l) % modular) * ((proving_key["zero_knowledge"]["beta_output_target_polynomial"] ** delta_r) % modular) * ((proving_key["zero_knowledge"]["beta_output_target_polynomial"] ** delta_o) % modular)) % modular
  # prover_consistency_check_polynomial = (prover_consistency_check_polynomial * apply_values(prover_values, proving_key["beta_polynomials"])) % modular

  proof["prover_left_polynomial"] = prover_left_polynomial
  proof["prover_right_polynomial"] = prover_right_polynomial
  proof["prover_output_polynomial"] = prover_output_polynomial

  proof["prover_shifted_left_polynomial"] = prover_shifted_left_polynomial
  proof["prover_shifted_right_polynomial"] = prover_shifted_right_polynomial
  proof["prover_shifted_output_polynomial"] = prover_shifted_output_polynomial

  proof["prover_result_polynomial"] = prover_result_polynomial
  proof["prover_consistency_check_polynomial"] = prover_consistency_check_polynomial
  # proof["rem"] = rem

  return proof

In [89]:
prover_proof(proving_key, witness)

p [1, 3, 35, 9, 27, 30]
v [1, 3, 35, 9, 27, 30]


{'prover_consistency_check_polynomial': <__main__.Point_on_EC at 0x7fd796014190>,
 'prover_left_polynomial': <__main__.Point_on_EC at 0x7fd796010c10>,
 'prover_output_polynomial': <__main__.Point_on_EC at 0x7fd796007d90>,
 'prover_result_polynomial': <__main__.Point_on_EC at 0x7fd796088590>,
 'prover_right_polynomial': <__main__.Point_on_EC at 0x7fd796007350>,
 'prover_shifted_left_polynomial': <__main__.Point_on_EC at 0x7fd796010090>,
 'prover_shifted_output_polynomial': <__main__.Point_on_EC at 0x7fd7960103d0>,
 'prover_shifted_right_polynomial': <__main__.Point_on_EC at 0x7fd796088790>}

In [90]:
for key, value in prover_proof(proving_key, witness).items():
  print(value.x, value.y)

p [1, 3, 35, 9, 27, 30]
v [1, 3, 35, 9, 27, 30]
477 409
214 507
467 77
305 506
67 481
497 49
56 105
182 458


# Verification

In [91]:
# proof = prover_proof(proving_key)

In [92]:
def variable_polynomials_restriction_check(proof, verification_key):

  if E.weil_pairing(proof["prover_left_polynomial"], verification_key["alpha_left"]) != E.weil_pairing(proof["prover_shifted_left_polynomial"],verification_key["generator"]):
    return False
  
  if E.weil_pairing(proof["prover_right_polynomial"], verification_key["alpha_right"]) != E.weil_pairing(proof["prover_shifted_right_polynomial"], verification_key["generator"]):
    return False
  
  if E.weil_pairing(proof["prover_output_polynomial"], verification_key["alpha_output"]) != E.weil_pairing(proof["prover_shifted_output_polynomial"], verification_key["generator"]):
    return False
  
  return True

In [93]:
def variable_values_consistency_check(proof, verification_key):
  result = E.addition(E.addition(proof["prover_left_polynomial"], proof["prover_right_polynomial"]), proof["prover_output_polynomial"])
  
  if E.weil_pairing(result, verification_key["beta_gamma"]) != E.weil_pairing(proof["prover_consistency_check_polynomial"], verification_key["gamma"]):
    return False
  
  return True

In [94]:
def valid_operations_check(proof, verification_key, verifier_left, verifier_right, verifier_output):
  l = E.addition(proof["prover_left_polynomial"], verifier_left)
  r = E.addition(proof["prover_right_polynomial"], verifier_right)
  o = E.addition(proof["prover_output_polynomial"], verifier_output)
  # l = (proof["prover_left_polynomial"] * verifier_left) % modular
  # r = (proof["prover_right_polynomial"] * verifier_right) % modular
  # o = (proof["prover_output_polynomial"] * verifier_output) % modular
  # print('proof["prover_left_polynomial"]', proof["prover_left_polynomial"].x, proof["prover_left_polynomial"].y)
  # print('proof["prover_right_polynomial"]', proof["prover_right_polynomial"].x, proof["prover_right_polynomial"].y)
  # print('proof["prover_output_polynomial"]', proof["prover_output_polynomial"].x, proof["prover_output_polynomial"].y)
  # print('l',l.x, l.y)
  # print('r',r.x, r.y)
  # print('o',o.x, o.y)

  result_left = E.weil_pairing(verification_key["target_polynomials"], proof["prover_result_polynomial"])
  result_right = E.weil_pairing(o, verification_key["generator"])
  # print(o)
  # print('tp', verification_key["target_polynomials"].x, verification_key["target_polynomials"].y)
  # print('prp', proof["prover_result_polynomial"].x, proof["prover_result_polynomial"].y)
  # print('vg', verification_key["generator"].x, verification_key["generator"].y)
  # print(result_left)
  # print(result_right)

  if isinstance(result_left, complex) or isinstance(result_right, complex):
    result = E.complex_multiply(result_left, result_right)
  else:
    result = (result_left * result_right) % modular

  # print('rl', result_left)
  # print('rr', result_right)
  # print('res', result)
  # print('lr', E.weil_pairing(l, r))

  if E.weil_pairing(l, r) != result:
    return False
  
  return True

In [95]:
def verification(proof, verification_key):
  if proof == False:
    return False
    
  verifier_values = variables_values(u)[:3]

  verifier_left_polynomial = apply_values(verifier_values, verification_key["verifier_left_polynomials"])
  verifier_right_polynomial = apply_values(verifier_values, verification_key["verifier_right_polynomials"])
  verifier_output_polynomial = apply_values(verifier_values, verification_key["verifier_output_polynomials"])
  # print(verifier_left_polynomial.x, verifier_left_polynomial.y)
  # print(verifier_right_polynomial.x, verifier_right_polynomial.y)
  # print(verifier_output_polynomial.x, verifier_output_polynomial.y)

  print('Variable polynomials restriction check:', variable_polynomials_restriction_check(proof, verification_key))
  print('Variable values consistency check:     ', variable_values_consistency_check(proof, verification_key))
  print('Valid operations check:                ', valid_operations_check(proof, verification_key, verifier_left_polynomial, verifier_right_polynomial, verifier_output_polynomial))
  if not variable_polynomials_restriction_check(proof, verification_key):
    return False
  
  if not variable_values_consistency_check(proof, verification_key):
    return False

  if not valid_operations_check(proof, verification_key, verifier_left_polynomial, verifier_right_polynomial, verifier_output_polynomial):
    return False
  
  return True

#Main

In [96]:
proving_key, verification_key = setup()

In [97]:
proving_key

{'beta_polynomials': [<__main__.Point_on_EC at 0x7fd7960bc490>,
 'evaluation': [<__main__.Point_on_EC at 0x7fd7960bc510>,
 'prover_left_polynomials': [<__main__.Point_on_EC at 0x7fd796181690>,
 'prover_output_polynomials': [<__main__.Point_on_EC at 0x7fd7960bc7d0>,
 'prover_right_polynomials': [<__main__.O at 0x7fd7960bc850>,
 'prover_shifted_left_polynomials': [<__main__.Point_on_EC at 0x7fd7960ebed0>,
 'prover_shifted_output_polynomials': [<__main__.Point_on_EC at 0x7fd7960bc790>,
 'prover_shifted_right_polynomials': [<__main__.O at 0x7fd7960bc4d0>,
 'zero_knowledge': {'beta_left_target_polynomial': <__main__.Point_on_EC at 0x7fd7960bc810>,
  'beta_output_target_polynomial': <__main__.Point_on_EC at 0x7fd7960bcd10>,
  'beta_right_target_polynomial': <__main__.Point_on_EC at 0x7fd7960bcf10>,
  'left_target_polynomial': <__main__.Point_on_EC at 0x7fd796007ed0>,
  'output_target_polynomial': <__main__.Point_on_EC at 0x7fd7960bccd0>,
  'right_target_polynomial': <__main__.Point_on_EC at 

In [98]:
verification_key

{'alpha_left': <__main__.Point_on_EC at 0x7fd7960bc390>,
 'alpha_output': <__main__.Point_on_EC at 0x7fd7960bc3d0>,
 'alpha_right': <__main__.Point_on_EC at 0x7fd7960bcf50>,
 'beta_gamma': <__main__.Point_on_EC at 0x7fd7960b5050>,
 'gamma': <__main__.Point_on_EC at 0x7fd7960b5090>,
 'generator': <__main__.Point_on_EC at 0x7fd7993bdfd0>,
 'target_polynomials': <__main__.Point_on_EC at 0x7fd7960bcc10>,
 'verifier_left_polynomials': [<__main__.Point_on_EC at 0x7fd7960bcfd0>,
 'verifier_output_polynomials': [<__main__.O at 0x7fd7960bcf90>,
 'verifier_right_polynomials': [<__main__.Point_on_EC at 0x7fd7960bc910>,
  <__main__.O at 0x7fd7960bc750>]}

In [99]:
witness = 3
proof = prover_proof(proving_key, witness)
proof

p [1, 3, 35, 9, 27, 30]
v [1, 3, 35, 9, 27, 30]


{'prover_consistency_check_polynomial': <__main__.Point_on_EC at 0x7fd7960ba4d0>,
 'prover_left_polynomial': <__main__.Point_on_EC at 0x7fd7960ba5d0>,
 'prover_output_polynomial': <__main__.Point_on_EC at 0x7fd7960ba510>,
 'prover_result_polynomial': <__main__.Point_on_EC at 0x7fd7960bacd0>,
 'prover_right_polynomial': <__main__.Point_on_EC at 0x7fd7960ba0d0>,
 'prover_shifted_left_polynomial': <__main__.Point_on_EC at 0x7fd7960bad50>,
 'prover_shifted_output_polynomial': <__main__.Point_on_EC at 0x7fd7960ba610>,
 'prover_shifted_right_polynomial': <__main__.Point_on_EC at 0x7fd7960baed0>}

In [100]:
verified = verification(proof, verification_key)

Variable polynomials restriction check: True
Variable values consistency check:      True
Valid operations check:                 True


In [101]:
verified

True

In [102]:
a = [0, 1, 2, 3, 4]
for i in a:
  proof = prover_proof(proving_key, i)
  verified = verification(proof, verification_key)
  print(verified)

p [1, 0, 5]
v [1, 3, 35, 9, 27, 30]


IndexError: ignored

In [ ]:
for i in range(100):
  proof = prover_proof(proving_key, i)
  verified = verification(proof, verification_key)
  print(i, verified)